In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from functools import partial
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
def build_model2D_basic(train_X, train_y, val_X, val_Y,
                kernel, activation, padding, pooling, batch_normalization, drop_out, optimizer, learning_rate, batch_size, epochs, val_data=True): # hyperparameter
    
    DefaultConv2D = partial(layers.Conv2D, kernel_size=kernel, activation=activation, padding=padding)
    
    model = models.Sequential()
    
   
    model.add(DefaultConv2D(filters=64, input_shape=(X_train_gadf[0].shape[0], X_train_gadf[0].shape[1], X_train_gadf[0].shape[2])))
    model.add(DefaultConv2D(filters=128))
    if pooling=="max":
        model.add(layers.MaxPooling2D((2,2)))
    elif pooling=="average":
        model.add(layers.AveragePooling2D((2,2)))

    if batch_normalization==True:
        model.add(layers.BatchNormalization())

    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation=activation))
    #layers.Dense(128, activation="relu"),

    if drop_out != None: 
        model.add(layers.Dropout(drop_out))

    # Output layer
    layers.Dense(3, activation="softmax")

    if optimizer=="adam":
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer =="RMSprop":
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer=="SGD":
        optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
    
    # es = EarlyStopping(monitor='val_accuracy', mode='min', verbose=1, patience=10)
    # mcp = ModelCheckpoint("best_model", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='max')
  
    if val_data == True: 
        model.fit(train_X, train_y, 
                   validation_data=(val_X, val_y ),
                   epochs=epochs, 
                   batch_size=batch_size,
                   class_weight=weights,
                  
                ) #callbacks=[es, mcp]
        
    else: # used for retraining after cross validation
        model.fit(train_X, train_y, 
                   epochs=epochs, 
                   batch_size=batch_size,
                   class_weight=weights,
                ) #callbacks=[es, mcp]
   
    return model

In [ ]:
grid={"kernel": [(3,3),(5,5)],
      "activation":["relu", "sigmoid", "softmax"],
      "padding": ["SAME", "VALID"],
      "pooling": ["max", "average"], 
      "batch_normalization": [True, False],
      "drop_out": [None, 0.25, 0.5],
      "epochs": [5, 10, 15, 20, 25, 30],
      "batch_size": [1,2,5,10,20,50,100],
      "optimizer": ["RMSprop",  "Adam", "SGD"], 
      "learning_rate": [0.0001, 0.001, 0.01] }


keys, values = zip(*grid.items())
permut_grid = [dict(zip(keys, v)) for v in itertools.product(*values)]

np.random.RandomState(100)
random.shuffle(permut_grid) # randomize order of parameter composition

In [ ]:
# cv splits

val_iterations=3
validation_size=250
max_train_val = len(y_train)-validation_size

val_split=np.arange(0, max_train_val, int(max_train_val/(val_iterations+1)))